# Week 6 - Linear Regression

## Learning Objectives
+ Introduction to Statsmodel
+ Implementing simple linear regression
+ Estimating coefficients and analyzing the model
+ Multiple Linear Regression and its difference with SLR

For this tutorial, you would need statsmodel installed on your system.

The tutorial is using the case study covered in the  textbook "An introduction to statistical learning: with applications in R" by James G. et al. ([link](https://link-springer-com.libproxy1.nus.edu.sg/book/10.1007/978-1-4614-7138-7)). The codes are written using this [notebook](https://github.com/justmarkham/DAT4/blob/master/notebooks/08_linear_regression.ipynb) and this [notebook](https://nbviewer.jupyter.org/github/JWarmenhoven/ISL-python/blob/master/Notebooks/Chapter%203.ipynb).

# Dataset

The Advertising data set consists of the sales of that product in 200 different markets, along with advertising budgets for the product in each of those markets for three different media: TV, radio, and newspaper. Say, you are a data analyst hired to provide advice on how to improve sales of a particular product.

It is not possible for your client to directly increase sales of the product. On the other hand, they can control the advertising expenditure in each of the three media. Therefore, if you determine that there is an association between advertising and sales, then you can instruct your client to adjust advertising budgets, thereby indirectly increasing sales.
In other words, your goal is to develop an **accurate model that can be used to predict sales on the basis of the three media budgets**.

The sales is in thousands of units and advertisement budges are in thousands of dollars.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('seaborn-white')

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

In [ ]:
advertising = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IT5006/Week 6/Advertising.csv', index_col=0)
advertising.describe()

In [ ]:
advertising.head()

# Introduction to Statsmodels

As the name implies, statsmodels is a Python library built specifically for statistics. Statsmodels is built on top of NumPy, SciPy, and matplotlib, but it contains more advanced functions for statistical testing and modeling that you won’t find in numerical libraries like NumPy or SciPy. To know more about this package follow the tutorials in this [link](https://www.statsmodels.org/stable/user-guide.html).

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

```Statsmodels``` allows users to fit statistical models using R-style formulas too. The ```formula.api``` hosts many of the same functions found in ```api``` (e.g. OLS, GLM), but it also holds lower case counterparts for most of these models. In general, lower case models accept ```formula``` and ```df``` arguments, whereas upper case ones take ```endog``` and ```exog``` design matrices.

# Simple Linear Regression

It predicts a response *Y* on the basis of a single predictor variable *X*. It assumes an approximately linear relationship between *X* and *Y*. 

We can regress sales onto TV by fitting the model:
$sales \approx \beta_{0} + \beta_{1} \times TV$

Before making predictions, we must data to estimate the coefficients. Let us plot the least squares fit for the regression of sales onto TV.

In [ ]:
sns.regplot(advertising.TV, advertising.Sales, order=1)
plt.ylim(ymin=0);
plt.show()

We demonstrate two ways to perform the regression:

In [ ]:
X = advertising[['TV']]
y = advertising[['Sales']]

est = sm.OLS(y, X).fit() 
est.summary()

In [ ]:
est = smf.ols('Sales ~ TV', advertising).fit()
est.summary()

## Residuals and assessing accuracy of model

### Residual and RSS

**Residual** ($e_{i} = y_{i}-\hat{y_{i}}$) is the difference between the $i^{th}$ observed response value and the $i^{th}$ response value that is predicted by our linear model. The least squares approach chooses the $\hat{\beta_{0}}$ and $\hat{\beta_{1}}$ to minimize the **Residual sum of squares** ($ e^{2}_{1} + e^{2}_{2} + ...$).

From the results obtained above, we can see the coefficient estimates as follows:
+ $\hat{\beta_{0}} = 7.03$
+ $\hat{\beta_{1}} = 0.0475$

In other words, an additional 1000 dollars spent on TV advertising is associated with selling approximately 47.5 additional units of the product. This is because the $\beta_{1}$ is the slope - the average increase in sales associated with a one-unit
increase in TV budget (which is in thousands of dollars).

In [ ]:
est.params

In [ ]:
(est.params[0] + est.params[1]*advertising.TV)
# RSS with regression coefficients

In [ ]:
est.ssr

Besides the summary presented above, the [```OLSResults```](https://www.statsmodels.org/devel/generated/statsmodels.regression.linear_model.OLSResults.html) class in ```statsmodels``` has various attributes which can be accessed.


### Residual Standard Error

Here we note that we have access to a set of observations from which we can compute the least squares line; however, the population regression line is unobserved - which would be the "true" model. To conduct hypothesis testing and to find the confidence intervals of our coefficients, we need to find the residual standard error. 

The residual sum of squares can be used to compute the residual standard error which is given by the formula:
$RSE = \sqrt{\frac{RSS}{n-2}}$ ([link](https://https://hastie.su.domains/ISLR2/ISLRv2_website.pdf))





In [ ]:
print("Residual standard error:", np.sqrt(est.ssr/(len(advertising.TV)-2)))

The RSE is found to be 3.26. In other words, actual sales in each market deviate from the true regression line by approximately 3.26 units, on average.

The residual standard error can be used to calculate the standard error of coefficients in the regression. These can then be used to compute hypothesis tests such as whether there is a relationship between X and Y (implying $H0: \beta_{1} = 0$). We need to do the following:
+ Calculate the standard error for each coefficient
+ Calculate the t test value for a 95% confidence interval
+ Calculate the confidence interval for each coefficient

The confidence interval is: $\frac{RSE}{\sqrt{\sum{(y_i - \overline{y_i})^2}}}$

In [ ]:
np.sqrt(est.ssr/(len(advertising.TV)-2)) / np.sqrt(np.sum((advertising.TV - np.mean(advertising.TV))**2))

In [ ]:
est.bse

In [ ]:
from scipy.stats import t
tvalue = t.ppf(1-0.025, df = 198)
tvalue

In [ ]:
print("Confidence interval for intercept", est.params[0] - est.bse[0] * tvalue, "to", est.params[0] + est.bse[0] * tvalue)
print("Confidence interval for TV coefficient", est.params[1] - est.bse[1] * tvalue, "to", est.params[1] + est.bse[1] * tvalue)

In [ ]:
est.conf_int(alpha = .05)

Alternatively, in the computed OLS output, we observe that $\hat{\beta_{0}}$ and $\hat{\beta_{1}}$ are very large relative to their
standard errors, so the t-statistics are also large; the probabilities of seeing such values if H0 is true are virtually zero. 

We can reject the null hypothesis that $\beta_{0} = 0$, which leads us to conclude that in the absense of TV expenditure, the sales are non-zero. 


We can also reject the null hypothesis that $\beta_{1} = 0$, which leads us to conclude that for each $1,000 increase in television advertising, there will be an average increase in sales of between 42 and 53 units. 

$\beta_{0}$ and $\beta_{1}$ are *statistically significant*.


In [ ]:
est.tvalues

### R-squared statistic
The $R^{2}$ statistic provides an alternative measure of fit. It takes the form of a proportion—the proportion of variance
explained—and so it always takes on a value between 0 and 1, and is independent of the scale of Y.

$ R^{2} = 1 - \frac{RSS}{TSS}$

TSS measures the total variance in the response Y , and can be squares thought of as the amount of variability inherent in the response before the regression is performed. In contrast, RSS measures the amount of variability that is left unexplained after performing the regression. An $R^{2}$ statistic that is close to 1 indicates that a large proportion of the variability in the response has been explained by the regression.

In [ ]:
print('R-squred: ', est.rsquared)

Now let us plot the residuals with respect to the predicted values.  

For this plot, we can use [```sns.residplot```](https://seaborn.pydata.org/generated/seaborn.residplot.html) which will regress y on x and then draw a scatterplot of the residuals. So the graph we will obtain will be a scatterplot for the residuals with the predictor variable.

In [ ]:
sns.residplot(advertising["TV"], advertising['Sales'], lowess=True, order=1)

 Ideally, this should look like a random scatter plot with zero mean and constant variance for all predicted values. Our residual plot doesn't look like an ideal residual plot. This is because the mean line of the residuals is not close to zero for all values of  $\hat{y}$ and the variance seems to be increasing with the predictor, TV. This means that our model assumptions are being violated.
 
Among many possible reasons for this violation, the following are usual suspects:
+ There is another feature other than TV that affects sales.
+ The relation between TV and sales is not linear. 

We will implement the first fix using Multiple Linear Regression. But first let's break the model linearity assumption.

# Polynomial Regression and Heteroscedasticity

As we see, the assumption of linearity between TV and sales might not hold. However, from the residual plot, we observe more more clearly that the standard deviation of residual keeps on increasing as the magnitude of predicted response increases. This issue is called **heteroscedasticity**. To address this issue, we could transform our response variable y with functions such as $\sqrt y$, $\log(y)$. We use the concept of indicator function denoted by $I(.)$ that returns a 1 if the condition is true, otherwise returns a 0. It represents a new variable, $TV^2$

In [ ]:
poly_est = smf.ols('np.log(Sales) ~ TV + I(TV**2)', data=advertising).fit()
poly_est.summary()

To see the values in the model:

In [ ]:
smf.ols('np.log(Sales) ~ TV + I(TV**2)', data=advertising).exog[:5,]

In [ ]:
sns.residplot(advertising["TV"], np.log(advertising['Sales']), lowess=True, order=2)

# Performing model prediction

Let us say, the client invests 50,000 in TV budget. What would we expect our sales to be? We can use the multiple linear regression model we have estimated to do the prediction for us. 

NOTE: The sklearn library also has implementation of linear regression which can be used for such prediction. The API is the same one you are already familiar with - the ```fit```, ```transform``` and ```fit_transform``` methods for the ```sklearn.linear_model.LinearRegression```.

In [ ]:
X_test = pd.DataFrame({'TV': [50]})
est.predict(X_test)

Linear models rely upon a lot of assumptions (such as the features being independent), and if those assumptions are violated (which they usually are), R-squared and p-values are less reliable. R-squared will always increase as you add more features to the model, even if they are unrelated to the response. Thus, selecting the model with the highest R-squared is not a reliable approach for choosing the best linear model. There is alternative to R-squared called adjusted R-squared that penalizes model complexity (to control for overfitting).

In general, to select the best model to use for prediction, you might want to resort to the classical cross validation. This is again easily possible using the ```sklearn``` API.

```statsmodels``` provides a function [```wls_prediction_std```](https://www.kite.com/python/docs/statsmodels.graphics.regressionplots.wls_prediction_std) that takes the result object of regression and provides the lower and upper values within which the prediction will lie with 95% confidence.

In [ ]:
from statsmodels.sandbox.regression.predstd import wls_prediction_std

def get_vis_dataframe(est, X, y, ylabel='yobs', yhlabel='ypred'):
    std, upper,lower = wls_prediction_std(est)
    rvis = X.copy()
    rvis[ylabel], rvis[yhlabel]= y, est.predict(X)
    rvis['resid'], rvis['upper'], rvis['lower'] = est.resid, upper, lower
    rvis2 = pd.melt(rvis[['TV', 'ypred', 'upper', 'lower']], id_vars = 'TV', var_name='Estimate') # massage the df into the format we want
    return rvis, rvis2

In [ ]:
linvis, linvis2 = get_vis_dataframe(est, X, y)
linvis.head()

In [ ]:
linvis2

In [ ]:
fig, ax = plt.subplots()

sns.scatterplot(data=linvis, x='TV', y='yobs', ax=ax)

# Multiple Linear Regression

When we have more than one predictor variables, we use multiple linear regression. It essentially gives each predictor a separate slope coefficient in a single model. So, if we want to analyze if the radio and newspapers are also associated with sales, we can do a multiple linear regression as:

$sales \approx \beta_{0} + \beta_{1} \times TV + \beta_{2} \times radio + \beta_{3} \times newspaper$

Let us first do simple linear regression on these features.

In [ ]:
est = smf.ols('Sales ~ Radio', advertising).fit()
est.summary().tables[1]

In [ ]:
est = smf.ols('Sales ~ Newspaper', advertising).fit()
est.summary().tables[1]

From the simple linear regression, we can say that a 1000 dollars increase in spending on radio advertising is associated with an average increase in sales by around 203 units, while the same increase in spending on newspaper advertising is associated with an average increase in sales by around 55 units.

In [ ]:
mul_est = smf.ols(formula='Sales ~ TV + Radio + Newspaper', data=advertising).fit()

# print the coefficients
mul_est.params

In [ ]:
print(mul_est.summary())

We notice that the multiple regression coefficient estimates for TV and Radio are pretty similar to the simple linear regression coefficients. However, while the newspaper regression coefficient estimate in SLR was significantly non-zero, the coefficient estimate for newspaper in the multiple regression model is close to zero, and the corresponding p-value is no longer significant, with a value around 0.86.

This difference stems from the fact that in the simple regression case, the slope term represents the average effect of a 1000 dollars increase in newspaper advertising, ignoring other predictors such as TV and radio. In contrast, in the multiple regression setting, the coefficient for newspaper represents the average effect of increasing newspaper spending by 1000 dollars while holding TV and radio fixed.

In [ ]:
advertising.corr()

From the correlation matrix, we notice that the correlation between radio and newspaper is 0.35. This reveals a tendency to spend more on newspaper advertising in markets where more is spent on radio advertising. 

Now suppose that the multiple regression is correct and newspaper advertising has no direct impact on sales, but radio advertising does increase sales. Then in markets where we spend more on radio our sales will tend to be higher, and as our correlation matrix shows, we also tend to spend more on newspaper advertising in those same markets. Hence, in a simple linear regression which only examines sales versus newspaper, we will observe that higher values of newspaper tend to be associated with higher values of sales, *even though newspaper advertising does not actually affect sales*. So newspaper sales are a surrogate for radio advertising; newspaper gets “credit” for the effect of radio on sales.

## F-statistic

The F value is the ratio of the mean regression sum of squares divided by the mean error sum of squares. Its value will range from zero to an arbitrarily large number. In multiple linear regression, the null hypothesis we look at is if there is a relationship between the response and predictor.

+ H0: $\beta_{1} = \beta_{2} = \beta_{3} = ... = 0$

We can assess the accuracy of the multiple regression model.

In [ ]:
print("RSE:", mul_est.resid.std())
print("R squared:", mul_est.rsquared)
print("F-statistic:", mul_est.fvalue)
print("F-test pvalue:", mul_est.f_pvalue)

The large F-statistic suggests that *at least one of the advertising media must be related to sales* (at least one is non-zero). We see that the probability of the f-statistic is close to zero, so we have extremely strong evidence that at least one of the media is associated with increased sales.

Suppose our null hypothesis is that specific coefficients are zero. The t-statistic for each predictor provide information about whether each individual predictor is related to the response, after adjusting for the other
predictors.

## Residual Plots for the multiple linear regression

Let us again plot the residual plots to visualize the trend in residuals as we did for simple linear regression. As the residplot function is not designed for the multiple linear regression, but only for simple linear regression, we will use the get_vis_dataframe function to get predictions from the model to plot.

In [ ]:
X = advertising[["TV", "Radio", "Newspaper"]]
y = advertising[["Sales"]]

mulvis, mulvis2 = get_vis_dataframe(mul_est, X, y)

sns.residplot(mulvis["ypred"], mulvis.resid, lowess=True, order=1)

In [ ]:
predictors = ["TV", "Radio", "Newspaper"]
poly2_predictors = []
for p in predictors:
    poly2_predictors.append('{} + I({}**2)'.format(p, p))
print(poly2_predictors)

In [ ]:
poly_mul_est = smf.ols(formula='Sales ~'+ ' + '.join(poly2_predictors), data=advertising).fit()
print(poly_mul_est.summary())

We plot the residual plot of the multiple polynomial regression.

In [ ]:
mulvis, mulvis2 = get_vis_dataframe(poly_mul_est, X, y)

sns.residplot(mulvis["ypred"], mulvis["yobs"], lowess=True)

This plot is equivalent to:

In [ ]:
plt.scatter(mulvis.ypred, mulvis.resid )

We also plot the residual plot of the multiple linear regression. Will the residual plot with ```order = 2``` be the same as that of the polynomial regression above?

In [ ]:
mulvis, mulvis2 = get_vis_dataframe(mul_est, X, y)
sns.residplot(mulvis["ypred"], mulvis["yobs"], lowess=True, order=2)

Why the line is straight when using order=2 in resid plot, while our regression model does not yield the same plot despite taking second degree terms?